In [10]:
import cv2
import numpy as np
from djitellopy import Tello

In [11]:
# 배터리 체크
def battery_check() : 
  drone = Tello()
  drone.connect()

  power = drone.get_battery()
  if power < 30 : print("배터리 부족", power)
  else : print("배터리", power)
  drone.end()
  return power
battery_check()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'


배터리 83


83

Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\Users\nono\anaconda3\envs\test\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\nono\anaconda3\envs\test\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\nono\anaconda3\envs\test\lib\site-packages\djitellopy\tello.py", line 1065, in update_frame
    self.stop()
  File "c:\Users\nono\anaconda3\envs\test\lib\site-packages\djitellopy\tello.py", line 1074, in stop
    self.worker.join()
  File "c:\Users\nono\anaconda3\envs\test\lib\threading.py", line 1008, in join
    raise RuntimeError("cannot join current thread")
RuntimeError: cannot join current thread


In [13]:
# 스타트flag
startCounter = 1

# 임계값 조절
tolerance_x = 5
tolerance_y = 5
# 속도 한계값
slowdown_threshold_x = 10
slowdown_threshold_y = 10
# 속도
drone_speen_x = 10
drone_speen_y = 10
# 화면 조절
set_point_x = 960/2
set_point_y = 720/2

# 드론 초기화
drone = Tello()
drone.connect()

# 스트림 연결 오류 
drone.streamon()

# 모델 임포트
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
while True :
  # takeoff
  if startCounter == 0 :
    drone.takeoff()
    print('takeoff')
    startCounter = 1

  # 프레임 읽어오기
  frame = drone.get_frame_read().frame
  # 읽은 프레임 회색으로 스케일 변환
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

  # 얼굴 검출
  faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minSize=(30,30), flags=cv2.CASCADE_SCALE_IMAGE)
  i = 0 

  for (x, y, w, h) in faces:
    # 인식 표시
    cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 2) # 인식얼굴 표시
    cv2.circle(frame, (int(x+w/2) , int(y+h/2)), 12, (255,0,0), 1) # 얼굴 중심 표시

    cv2.circle(frame, (int(set_point_x), int(set_point_y)), 12, (255,255,0), 3) # 화면 중간에 원표시
    i +=1

    #  얼굴중심과 화면중심의 차를 계산
    distance_x = x+w/2 - set_point_x
    distance_y = y+h/2 - set_point_y

    up_down_velocity = 0
    right_left_veiocity = 0

# 드론 좌우 이동
    if distance_x < -tolerance_x:
      print("left move")
      right_left_veiocity = - drone_speen_x
    elif distance_x > tolerance_x:
      print("right move")
      right_left_veiocity = drone_speen_x
    else :
      print("OK")

  # 드론 상하 이동
    if distance_y < -tolerance_y:
      print("up move")
      up_down_velocity = drone_speen_y
    elif distance_y > tolerance_y:
      print("down move")
      up_down_velocity = - drone_speen_y
    else :
      print("OK")

#  임계치 이상 벗어나면 속도 조정 
    if abs(distance_x) < slowdown_threshold_x:
      right_left_veiocity = int(right_left_veiocity / 2)
    if abs(distance_y) < slowdown_threshold_y:
      up_down_velocity = int(up_down_velocity / 2)

    # 드론 움직이기
    drone.send_rc_control(right_left_veiocity, 0, up_down_velocity, 0)
  
  cv2.imshow("Video", frame)

  key = cv2.waitKey(1)
  if key == ord('q'):
    break

cv2.destroyAllWindows()
drone.streamoff()
drone.end()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'ok'


Exception: Failed to grab first frame from video stream

In [12]:
# 스타트flag
startCounter = 1

# 임계값 조절
tolerance_x = 3
tolerance_y = 3
# 속도 한계값
slowdown_threshold_x = 15
slowdown_threshold_y = 15
# 속도
drone_speen_x = 10
drone_speen_y = 10
# 화면 조절
set_point_x = 960/2
set_point_y = 720/2

# 드론 초기화
drone = Tello()
drone.connect()

# 스트림 연결 오류 
drone.streamon()

# 모델 임포트
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
while True :
  frame = drone.get_frame_read().frame
  # takeoff
  if startCounter == 0 :
    drone.takeoff()
    print('takeoff')
    startCounter = 1

  # 프레임 읽어오기
  frame = drone.get_frame_read().frame

  img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

  img_hsv = cv2.fastNlMeansDenoisingColored(img_hsv,None,10,10,7,21)
  lower = np.array([0,48,80], dtype="uint8")
  upper = np.array([20,255,255], dtype="uint8")
  img_hand = cv2.inRange(img_hsv,lower,upper)
  
  #경계선 찾음
  contours, hierarchy = cv2.findContours(img_hand, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  # 가장 큰 영역 찾기
  max = 0
  maxcnt = None
  for cnt in contours :
    area = cv2.contourArea(cnt)
    if(max < area) :
      max = area
      maxcnt = cnt
      if maxcnt.all() == None:
        break
  #print(maxcnt)
  
  mask = np.zeros(frame.shape).astype(frame.dtype)
  # 경계선 내부 255로 채우기
  color = [255, 255, 255]
  cv2.fillPoly(mask, [maxcnt], color)
  img_hand = cv2.bitwise_and(frame, mask)
  hull = cv2.convexHull(maxcnt)

  # 도출된 값으로 사각형 그리기
  x,y,w,h = cv2.boundingRect(hull)
  cv2.rectangle(mask, (x,y), (x+w,y+h), (0,255,0),3 )
  cv2.circle(frame, (int(x+w/2) , int(y+h/2)), 12, (255,0,0), 1) # 얼굴 중심 표시


  # 읽은 프레임 회색으로 스케일 변환
  # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

  # 얼굴 검출
  # faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minSize=(30,30), flags=cv2.CASCADE_SCALE_IMAGE)
  # i = 0 

#   for (x, y, w, h) in faces:
#     # 인식 표시
#     cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 2) # 인식얼굴 표시
#     cv2.circle(frame, (int(x+w/2) , int(y+h/2)), 12, (255,0,0), 1) # 얼굴 중심 표시

#     cv2.circle(frame, (int(set_point_x), int(set_point_y)), 12, (255,255,0), 3) # 화면 중간에 원표시
#     i +=1

#     #  얼굴중심과 화면중심의 차를 계산
#     distance_x = x+w/2 - set_point_x
#     distance_y = y+h/2 - set_point_y

#     up_down_velocity = 0
#     right_left_veiocity = 0

# # 드론 좌우 이동
#     if distance_x < -tolerance_x:
#       print("left move")
#       right_left_veiocity = - drone_speen_x
#     elif distance_x > tolerance_x:
#       print("right move")
#       right_left_veiocity = drone_speen_x
#     else :
#       print("OK")

#   # 드론 상하 이동
#     if distance_y < -tolerance_y:
#       print("up move")
#       up_down_velocity = drone_speen_y
#     elif distance_y > tolerance_y:
#       print("down move")
#       up_down_velocity = - drone_speen_y
#     else :
#       print("OK")

# #  임계치 이상 벗어나면 속도 조정 
#     if abs(distance_x) < slowdown_threshold_x:
#       right_left_veiocity = int(right_left_veiocity / 2)
#     if abs(distance_y) < slowdown_threshold_y:
#       up_down_velocity = int(up_down_velocity / 2)

    # 드론 움직이기
    # drone.send_rc_control(right_left_veiocity, 0, up_down_velocity, 0)


  cv2.imshow("Video", frame)

  key = cv2.waitKey(1)
  if key == ord('q'):
    break

cv2.destroyAllWindows()
drone.streamoff()
drone.end()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'ok'
[INFO] tello.py - 437 - Send command: 'streamoff'
[INFO] tello.py - 461 - Response streamoff: 'ok'


Exception: Failed to grab first frame from video stream